# yaml

> Load YAML strings into Pydantic models with nicer validation errors

[![](https://github.com/hamelsmu/pydantic-yaml-parser/actions/workflows/test.yaml/badge.svg)](https://github.com/hamelsmu/pydantic-yaml-parser/actions/workflows/test.yaml)

In [ ]:
#| default_exp yaml

In [ ]:
#| export
import sys
from pydantic import BaseModel, ValidationError
import yaml
from typing import List, Dict
from fastcore.test import test_fail

In [ ]:
yaml_string ="""
quartodoc:
  style: pkgdown
  dir: api
  package: quartodoc
  sidebar: "api/_sidebar.yml"
  sections:
    - title: Preperation Functions
      desc: |
        These functions fetch and analyze python objects, including parsing docstrings.
        They prepare a basic representation of your doc site that can be rendered and built.
      contents:
        - Auto
        - blueprint
        - collect
        - get_object
        - preview
"""

In [ ]:
#|export
def fmt(err:dict):
    "format error messages from pydantic."
    msg = ""
    if err['type'] == 'value_error.missing':
        msg += 'Missing field'
    else:
        msg += err['msg'] + ':'
        
    if 'loc' in err:
        if len(err['loc']) == 1:
            msg += f" from root level: `{err['loc'][0]}`"
        elif len(err['loc']) == 3:
            msg += f" `{err['loc'][2]}` for element {err['loc'][1]} in the list for `{err['loc'][0]}`"
                
    else:
        msg += str(err['msg'])
    return msg

In [ ]:
#|export
def yaml2d(yml:str) -> dict:
    "Turn a yaml string into a dict"
    return yaml.safe_load(yml)

In [ ]:
#|export
class Section(BaseModel):
    title: str
    desc: str
    contents: List[str]


class YamlModel(BaseModel):
    @classmethod
    def from_dict(cls, ymldict:dict, f:callable=fmt):
        sys.tracebacklimit = 0
        try:
            return cls.parse_obj(ymldict)
        except ValidationError as e:
            if f:
                msg = 'Configuration error(s) for YAML:\n - '
                msg += '\n - '.join(f(err) for err in e.errors())           
                raise ValueError(msg) from None
            else: 
                raise e

class QuartoDoc(YamlModel):
    style: str
    dir: str
    package: str
    sidebar: str
    sections: List[Section]

`QuartoDoc` has a `from_dict` method so that you can load a dict that corresponds to a yaml file into a Pydantic data model:

In [ ]:
yaml_dict = yaml2d(yaml_string)['quartodoc']
QuartoDoc.from_dict(yaml_dict)

QuartoDoc(style='pkgdown', dir='api', package='quartodoc', sidebar='api/_sidebar.yml', sections=[Section(title='Preperation Functions', desc='These functions fetch and analyze python objects, including parsing docstrings.\nThey prepare a basic representation of your doc site that can be rendered and built.\n', contents=['Auto', 'blueprint', 'collect', 'get_object', 'preview'])])

## Error Validation

### Missing Section

In the below yaml, there are two things missing: 


- The `contents` field is missing from `sections`.
- The root of the quartodoc config is missing a `dir` field.

In [ ]:
invalid_section = """
quartodoc:
  style: pkgdown
  package: quartodoc
  sidebar: "api/_sidebar.yml"
  sections:
    - title: Preperation Functions
      desc: |
        These functions fetch and analyze python objects, including parsing docstrings.
        They prepare a basic representation of your doc site that can be rendered and built.
"""

yaml_dict = yaml2d(invalid_section)['quartodoc']

If we try to load the yaml in `invalid_section` we get the following error message:

In [ ]:
#|eval: false
QuartoDoc.from_dict(yaml_dict)

ValueError: Configuration error(s) for YAML:
 - Missing field from root level: `dir`
 - Missing field `contents` for element 0 in the list for `sections`

In [ ]:
#|hide
test_fail(QuartoDoc.from_dict, args=(yaml_dict,), 
          contains="Missing field from root level: `dir`")

test_fail(QuartoDoc.from_dict, args=(yaml_dict,), 
          contains="Missing field `contents` for element 0 in the list for `sections`")

### Invalid Type

In the below yaml we will erroneously set the `contents` field to `false`, when it is supposed to be a list:

In [ ]:
yaml_string ="""
quartodoc:
  style: pkgdown
  dir: api
  package: quartodoc
  sidebar: "api/_sidebar.yml"
  sections:
    - title: Preperation Functions
      desc: |
        These functions fetch and analyze python objects, including parsing docstrings.
        They prepare a basic representation of your doc site that can be rendered and built.
      contents: false
"""

yaml_dict = yaml2d(yaml_string)['quartodoc']

We get the following human readable error message:

In [ ]:
#|eval: false
yml = QuartoDoc.from_dict(yaml_dict)

ValueError: Configuration error(s) for YAML:
 - value is not a valid list: `contents` for element 0 in the list for `sections`

In [ ]:
#|hide
test_fail(QuartoDoc.from_dict, args=(yaml_dict,), 
          contains="- value is not a valid list: `contents` for element 0 in the list for `sections`")

In the below yaml we errenously set `dir` to a list when it should be a `str`:

In [ ]:
yaml_string ="""
quartodoc:
  style: pkgdown
  dir:
      - folder1
      - folder2
  package: quartodoc
  sidebar: "api/_sidebar.yml"
  sections:
    - title: Preperation Functions
      desc: |
        These functions fetch and analyze python objects, including parsing docstrings.
        They prepare a basic representation of your doc site that can be rendered and built.
      contents:
          - item 1
          - item 2
"""

yaml_dict = yaml2d(yaml_string)['quartodoc']

In [ ]:
#|eval: false
yml = QuartoDoc.from_dict(yaml_dict)

ValueError: Configuration error(s) for YAML:
 - str type expected: from root level: `dir`

In [ ]:
#|hide
test_fail(QuartoDoc.from_dict, args=(yaml_dict,), 
          contains="- str type expected: from root level: `dir`")